In [ ]:
import numpy as np
import jax
import jax.numpy as jnp
import numpyro
import numpyro.distributions as dist
from numpyro.infer import MCMC, NUTS
from numpyro.diagnostics import hpdi

import patsy
import seaborn as sns
import matplotlib.pyplot as plt
import arviz as az
from GLM import models,datasim,glm

Make a data simulator

In [ ]:
sims=datasim.simulate_poisson_grouped()
X_dsgn=sims[0]
Y=sims[1]

In [ ]:
#Make GLM class
# FIT WITH INTERACTIONS
mod2fitall = glm.PoissonGLMbayes()

mod2fitall.add_data(y=jnp.array(Y))

# Learn smoothness from data
mod2fitall.define_model(model='grouped_ard', basis_x_list=X_dsgn, S_list=None,
                          tensor_basis_list=None, S_tensor_list=None)

paramglm = {'fittype': 'vi', 'guide': 'normal', 'visteps': 10000, 'optimtype': 'scheduled'}
mod2fitall.fit(params=paramglm,  fit_intercept=True, cauchy=0.1)

#Got to fix post sampling checking to beta_i * lambda_ard[i]
mod2fitall.sample_posterior(5000).summarize_posterior(90).coeff_relevance()
